In [1]:
import tensorflow as tf
from keras.models import load_model
#This is the nn.keras file completed in the "Lancaster_Encounter_Model" notebook. That will need to be uploaded alongside this python file to work.
model = load_model('nn.keras')

In [21]:
#  Import and read the encounter data.
import pandas as pd

#  Import and read the csv file.
path = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRlIMLpo7omYmZ-GtDVeu7eXivg1QF2DRr-jFG_sCt4YIQ1V9UJvq1ryg-3pF8sl0zXiE1cF4XSdH4W/pub?gid=678314375&single=true&output=csv'

sheriff_df = pd.read_csv(path)
# sheriff_df.head()

In [22]:
age = input('Enter your age: ')
english = input('Are you fluent in English (y/n)? ')
perception = 'After Contact'
homeless = input('Are you homeless (y/n)? ')
gender = input('What is your sex ("Male" or "Female")? ')
race = input('What is your race? \n Black/African American = 1) \n Hispanic/Lation/Latina = 2 \n White = 3 \n Mixed = 4 \n Middle Eastern/South Asian = 5 \n Asian = 6 \n Pacific Islander = 7 \n Native American = 8 \n Input here: ')


Enter your age: 35
Are you fluent in English (y/n)? y
Are you homeless (y/n)? n
What is your sex ("Male" or "Female")? Male
What is your race? 
 Black/African American = 1) 
 Hispanic/Lation/Latina = 2 
 White = 3 
 Mixed = 4 
 Middle Eastern/South Asian = 5 
 Asian = 6 
 Pacific Islander = 7 
 Native American = 8 
 Input here: 3


In [23]:
age = int(age)

if english == 'y':
  english = True
else:
  english = False

if homeless == 'y':
  homeless = True
else:
  homeless = False

race = int(race)
if race == 1:
  race = 'Black/African American'
elif race == 2:
  race = 'Hispanic/Lation/Latina'
elif race == 3:
  race = 'White'
elif race == 4:
  race = 'Mixed'
elif race == 5:
  race = 'Middle Eastern/South Asian'
elif race == 6:
  race = 'Asian'
elif race == 7:
  race = 'Pacific Islander'
elif race == 8:
  race = 'Native American'

In [24]:
user_input_arr = [age, english, perception, homeless, gender, race, 'No Disabilities', False]
user_input_arr

[35, True, 'After Contact', False, 'Male', 'White', 'No Disabilities', False]

In [25]:
sheriff_df.loc[len(sheriff_df.index)] = user_input_arr

In [26]:
sheriff_df['Race'] = sheriff_df['Race'].apply(lambda x: 'Mixed' if ',' in x else x)
sheriff_df['Gender'] = sheriff_df['Gender'].loc[(sheriff_df['Gender']=='Male') | (sheriff_df['Gender']=='Female')]
sheriff_df.drop(columns=['Disability'], inplace=True)
sheriff_df = sheriff_df.join(pd.cut(sheriff_df.pop('Age'),
                     bins=[1,10,15,20,25,30,35,40,45,50,55,60,120],
                     labels=['1-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60+'],
                     include_lowest=True,).to_frame('Age_Bins'))
sheriff_df = sheriff_df.loc[sheriff_df['Age_Bins']!='1-9']
sheriff_df['Age_Bins'] = sheriff_df['Age_Bins'].astype(str)

In [27]:
categories = sheriff_df.select_dtypes(include=['object']).columns.tolist()
sheriff_df_encoded = pd.get_dummies(sheriff_df, columns=categories)

from sklearn.preprocessing import StandardScaler

y_test = sheriff_df_encoded['Arrested']

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(sheriff_df_encoded.drop(columns='Arrested'))

# Scale the data
X_test_scaled = X_scaler.transform(sheriff_df_encoded.drop(columns='Arrested'))

In [28]:
user_input_test = X_test_scaled[len(X_test_scaled)-1]

In [30]:
predictions = model.predict(X_test_scaled)

120/120 [==============================] - 0s 2ms/step


In [42]:
pred_value = predictions[len(predictions)-1][0]

In [43]:
if pred_value > .5:
  print("You would likely be arrested by the Lancaster Sheriff's Office")
else:
  print("Lucky you. You would likely not have been arrested by the Lancaster Sheriff's Office.")

You would likely be arrested by the Lancaster Sheriff's Office
